# SEYFERT Demo Notebook

Imports

In [ ]:
import time

t_begin = time.time()

from pathlib import Path
import numpy as np
import importlib
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import datetime
import re
import sys
import copy
import json
import itertools
import pickle

from seyfert.utils import general_utils, filesystem_utils, formatters
from seyfert.utils.tex_utils import TeXTranslator

plt.style.use("plt_params.mplstyle")

from seyfert.config.forecast_config import ForecastConfig
from seyfert.utils.workspace import WorkSpace
from seyfert.cosmology import cosmology
from seyfert.cosmology import redshift_density
from seyfert.cosmology import c_ells

from seyfert.main import cl_core
from seyfert.main import cl_derivative_core
from seyfert.main import fisher_core

transl = TeXTranslator()

Logging: if you want full logs set do_full_log = True

In [ ]:
do_full_log = False

import logging
logger = logging.getLogger()

if do_full_log:
    general_utils.configure_logger(logger)

## Configurations

SEYFERT configuration files are all written in JSON format, and are essentially of two types: 

* ForecastConfig: the master configuration for the forecast. It is associated to a ForecastConfig class defined in `seyfert/config/forecast_config.py`
* MainConfig: the configuration for the main script do the single computation tasks, which are:
    * computation of the power spectra;
    * computation of the angular power spectra;
    * computation of the derivatives of the angular power spectra;
    * computation of the fisher matrices.

First of all we do the workspace setup. The necessary input data are:

* Configuration files: these are the above mentioned JSON files. Some example files are stored inside the directory `input/config`, and are written in the cell below.
* Input data files: these are auxiliary files storing the configuration of galaxy redshift densities or galaxy bias. The example files are here stored in `input/data`.

In [ ]:
ws = WorkSpace("rundir_test")
ws.run_dir.mkdir(exist_ok=True)

input_data_dir = Path("input/data/")

src_input_files = {
    'forecast': "input/config/basic_forecast_config.json",
    'PowerSpectrum': "input/config/power_spectrum_config.json",
    'Angular': "input/config/angular_config.json",
    'Derivative': "input/config/derivative_config.json",
    'Fisher': "input/config/fisher_config.json"
}

fcfg = ForecastConfig(input_file=src_input_files['forecast'], input_data_dir=input_data_dir)
fcfg.loadPhysicalParametersFromJSONConfig()
phys_pars = fcfg.phys_pars

ws.createInputFilesDir(src_input_files=src_input_files, phys_pars=phys_pars, input_data_dir=input_data_dir)

main_configs = ws.getTasksJSONConfigs()

## Fiducial Cosmology

In this forecast we work in a $w_0 w_a \rm CDM$ flat cosmology, therefore the Hubble parameter is given by

\begin{equation}
H(z)=H_{0} \sqrt{\Omega_{\mathrm{m}}(1+z)^{3}+\left(1-\Omega_{\mathrm{m}}\right)(1+z)^{3\left(1+w_{0}+w_{a}\right)} \exp \left(-3 w_{a} \frac{z}{1+z}\right)}
\end{equation}

In following cell the fiducial cosmology is loaded from file. Technically SEYFERT is also capable to call CAMB or CLASS to compute the matter power spectrum on the fly, but here in order to save time we load the already computed power spectra from disk.

In [ ]:
#pmm_dir = Path("/ssd860/euclid_forecasts/spectrophoto/powerspectra/istf_pmms/")
pmm_dir = Path("/Users/lucapaganin/spectrophoto/powerspectra/istf_pmms/")

if not ws.pmm_dir.exists():
    ws.symlinkToExternalDirs({
        "PowerSpectrum": pmm_dir
    })

fid_cosmo = cosmology.Cosmology.fromHDF5(ws.getPowerSpectrumFile(dvar='central', step=0))
fid_cosmo.evaluateOverRedshiftGrid()

## Redshift densities

Here the redshift densities are computed and saved to disk.

For $\rm GC_{ph}$ we use the redbook theoretical density:

\begin{equation}
\frac{\mathrm{d} N^{\mathrm{ph}}}{\mathrm{d} z \mathrm{~d} \Omega}(z)=N_{0}^{\mathrm{ph}}\left(\frac{z}{z_{0}}\right)^{2} \exp \left[-\left(\frac{z}{z_{0}}\right)^{3 / 2}\right]
\end{equation}

where $z_{0}=0.9 / \sqrt{2}$ and the normalization factor $N_{0}^{\mathrm{ph}}$ is chosen such that the surface densityof galaxies is equal to 30 galaxies per $\rm arcmin^2$.

For $\rm GC_{sp}$ we employ the data from SPV2 which have been provided to use by Ben Granett.

The *observed* galaxy densities are obtaining by "convolving" the theoretical distribution with a sort of gaussian probability (as done in IST:Forecast):

\begin{equation}
n_{i}^{A}(z)=
\frac{\int_{z_{i}^{-}}^{z_{i}^{+}} \mathrm{d} z_{\mathrm{p}} \frac{\mathrm{d} N^{A}}{\mathrm{d} z \mathrm{d} \Omega}(z) p_{A}(z_{\mathrm{p}} \mid z)}{\int_{z_{\min }}^{z_{\max }} \mathrm{d} z \int_{z_{i}^{-}}^{z_{i}^{+}} \mathrm{d} z_{\mathrm{p}} \frac{\mathrm{d} N^{A}}{\mathrm{d} z \mathrm{d} \Omega}(z) p_{A}(z_{\mathrm{p}} \mid z)}
\end{equation}

where $A$ stands for $\rm GC_{ph}, GC_{sp}$ and $i$ is the tomographic bin index. The probability $p\left(z_{\mathrm{p}} \mid z\right)$ is parameterized as:

\begin{equation}
p\left(z_{\mathrm{p}} \mid z\right)
=\frac{1-f_{\text {out }}}{\sqrt{2 \pi} \sigma_{\mathrm{b}}(1+z)} \exp \left\{-\frac{1}{2}\left[\frac{z-c_{\mathrm{b}} z_{\mathrm{p}}-z_{\mathrm{b}}}{\sigma_{\mathrm{b}}(1+z)}\right]^{2}\right\}+\frac{f_{\text {out }}}{\sqrt{2 \pi} \sigma_{\mathrm{o}}(1+z)} \exp \left\{-\frac{1}{2}\left[\frac{z-c_{0} z_{\mathrm{p}}-z_{0}}{\sigma_{\mathrm{o}}(1+z)}\right]^{2}\right\}
\end{equation}

The parameters used for $p\left(z_{\mathrm{p}} \mid z\right)$ are reported in the following table:

| probe                       |  $c_{\mathrm{b}}$    |  $z_{\mathrm{b}}$    |  $\sigma_{\mathrm{b}}$    |  $c_{\mathrm{o}}$    |  $z_{\mathrm{o}}$    |  $\sigma_{\mathrm{o}}$    |  $f_{\mathrm{out}}$   |
|:----------------------------|:---------------------|:---------------------|:--------------------------|:---------------------|:---------------------|:--------------------------|:----------------------|
| $\mathrm{GC}_{\mathrm{ph}}$ | $1.0$                | $0.0$                | $0.050$                   | $1.0$                | $0.1$                | $0.05$                    | $0.1$                 |
| $\mathrm{GC}_{\mathrm{sp}}$ | $1.0$                | $0.0$                | $0.001$                   | $-$                  | $-$                  | $-$                       | $-$                   |


Now let's code.

In [ ]:
if not ws.niz_file.is_file():
    densities = {
        "Lensing": redshift_density.RedshiftDensity.fromHDF5("input/data/WL/gcph_dndz_redbook.h5"),
        "PhotometricGalaxy": redshift_density.RedshiftDensity.fromHDF5("input/data/GCph/gcph_dndz_redbook.h5"),
        "SpectroscopicGalaxy": redshift_density.RedshiftDensity.fromHDF5("input/data/GCsp/gcsp_dndz_4_bins.h5")
    }
    for probe in densities:
        densities[probe].setUp()
        densities[probe].evaluate(z_grid=fid_cosmo.z_grid)
        densities[probe].evaluateSurfaceDensity()

    redshift_density.save_densities_to_file(densities=densities, file=ws.niz_file)
else:
    densities = redshift_density.load_densities_from_file(file=ws.niz_file)

### Photo-z densities

Here we plot the $\rm GC_{ph}$ galaxy densities. The tomographic bins used are 10 equi-populated bins in the range $z \in [0.001, 2.5]$. The bin edges are the following

\begin{equation}
z_{i}=\{0.0010,0.42,0.56,0.68,0.79,0.90,1.02,1.15,1.32,1.58,2.50\}
\end{equation}

In [ ]:
nph = densities["PhotometricGalaxy"]

for i in range(nph.n_bins):
    plt.plot(nph.z_grid, nph.norm_density_iz[i], label=f'{i+1}')

plt.xlabel("$z$", fontsize=22)
plt.legend(bbox_to_anchor=[1, 0.38], title="bin index")
plt.title(r"$\rm GC_{ph}$")
    
del nph

### Spectro-z densities

Here we plot the $\rm GC_{sp}$ galaxy densities. The baseline tomographic bins used for spectro-z are the 4 bins used for the power spectrum analysis of IST:F

\begin{equation}
z_{i} = \{0.9, 1.1, 1.3, 1.5, 1.8\}
\end{equation}

In [ ]:
nsp = densities["SpectroscopicGalaxy"]

for i in range(nsp.n_bins):
    plt.plot(nsp.z_grid, nsp.norm_density_iz[i], label=f'{i+1}')

plt.xlabel("$z$")
plt.legend(bbox_to_anchor=[1, 0.70], title="bin index")
plt.title(r"$\rm GC_{sp}$")
    
del nsp

## Angular power spectra

### Limber approximation

Here the $C(\ell)$'s in the reference cosmology are evaluated. The Limber approximation is used to compute the $C(\ell)$'s:

\begin{equation}
C^{AB}_{ij}(\ell) \simeq \frac{c}{H_0} \int_{z_\mathrm{min}}^{z_\mathrm{max}} \mathrm{d} z 
\frac{W_{i}^{A}(z) W_{j}^{B}(z)}{E(z) \chi^{2}(z)} P_{\delta\delta}\left[ k=\frac{\ell+1/2}{r(z)}, z \right]
\end{equation}

where $\chi(z)$ is the comoving distance and $E(z)$ is the dimensionless Hubble parameter. 

### Weight functions for galaxy clustering

For galaxy clustering the weight functions are defined by

\begin{equation}
W^{\mathrm{A}}_i(z) = b^{\mathrm{A}}(z) \frac{H(z)}{c} n^{\mathrm{A}}_{i}(z)\, \qquad \mathrm{A} = \mathrm{GC_{ph}}, \mathrm{GC_{sp}}
\end{equation}

The galaxy bias is assumed to be a step piecewise function. The photometric bias is computed as in IST:F

\begin{equation}
b^{\mathrm{ph}}(z) = b^{\mathrm{ph}}_i = \sqrt{1 + \bar{z}_i}, \quad \text{when} \quad z_i^- < z < z_i^+ \quad \text{and} \quad \bar{z}_i = \frac{z_i^- + z_i^+}{2}
\end{equation}

The spectroscopic bias is the same used for the IST:F Fourier power spectrum analysis. The bias values in the 4 spectroscopic bins are

\begin{equation}
b^{\mathrm{sp}}_i = \{1.46, 1.61, 1.75, 1.90 \}
\end{equation}

### Weight function for weak lensing

For cosmic shear the weight function is

\begin{equation}
W_i^\gamma(z) = 
\frac{3}{2}\left(\frac{H_{0}}{c}\right)^{2} \Omega_{\rm m}(1+z) \chi(z) \int_{z}^{z_{\rm max}} \mathrm{d} z'\, n^{\rm ph}_{i}(z')\left[1 - \frac{\chi(z)}{\chi(z')}\right]\, .
\end{equation}

Introducing the intrinsic alignment contribution we obtain the weak lensing weight function:

\begin{equation}
W_{i}^{\rm wl}(z) = W_{i}^{\gamma}(z) - \mathcal{A}_{\mathrm{IA}} C_{\mathrm{IA}} \Omega_{\rm m} \frac{H(z)\mathcal{F}_{\rm IA}(z)}{c D(z)} n^{\rm ph}_{i}(z)\, .
\end{equation}

In [ ]:
ws.cl_dir.mkdir(exist_ok=True)

fid_cl_file = ws.cl_dir / "dvar_central_step_0" / "cls_fiducial.h5"

if not fid_cl_file.is_file():

    fid_cls = cl_core.compute_cls(cosmology=fid_cosmo, phys_pars=phys_pars, densities=densities, 
                                  forecast_config=fcfg, angular_config=main_configs['Angular'])

    fid_cl_file.parent.mkdir(exist_ok=True, parents=True)

    fid_cls.saveToHDF5(fid_cl_file)
else:
    fid_cls = c_ells.AngularCoefficientsCollector.fromHDF5(fid_cl_file)

del fid_cl_file

### Plot the weight functions

Here we plot the weight functions

In [ ]:
from seyfert.plot_utils.cl_plot import plot_weight_funcs

In [ ]:
plot_weight_funcs(fid_cls, cosmo=fid_cosmo, phys_pars=phys_pars)

### Plot auto-correlation angular power spectra

Here we plot the diagonals of the auto-correlations $C(\ell)$'s

In [ ]:
for kind in ['phph', 'spsp', 'wlwl']:
    fig = plt.figure()
    cl = fid_cls[kind]
    ells = cl.l_bin_centers

    for i in range(cl.n_i):
        plt.loglog(ells, ells*(ells+1)*cl.c_lij[:, i, i], label=i)

    plt.legend(bbox_to_anchor=[1.0, 0.4], title="bin index")
    plt.title(r"$C^{\rm %s}_{ii}(\ell)$" % kind)
    plt.xlabel(r"$\ell$", fontsize=22)
    plt.ylabel(r"$\ell (\ell+1) C(\ell)$", fontsize=22)

## Derivatives of angular power spectra

Compute $C(\ell)$ derivatives with respect to the free parameters, which may be cosmological or nuisance. The derivatives are computed with the SteM® algorithm (credits to Stefano Camera).


First of all set the free parameters

In [ ]:
#free_pars = ["w0", "wa", "Omm"]
free_pars = list(phys_pars.free_physical_parameters.keys())

for name in phys_pars:
    if name not in free_pars:
        phys_pars[name].is_free_parameter = False
    else:
        phys_pars[name].is_free_parameter = True

Do the computation

In [ ]:
ws.der_dir.mkdir(exist_ok=True)
cl_ders_file = ws.der_dir / "cl_ders.pickle"

if not cl_ders_file.is_file():
    data = {
        "fid_cls": fid_cls,
        "ws": ws,
        "phys_pars": phys_pars,
        "densities": densities,
        "forecast_config": fcfg,
        "angular_config": main_configs['Angular'],
        "fiducial_cosmology": fid_cosmo
    }
    
    ti = time.time()
    ders_dict = {}
    n_params = len(free_pars)

    for i, dvar in enumerate(phys_pars.free_physical_parameters):
        t1 = time.time()
        print(f"{'#'*40} Computing cl derivatives w.r.t. {dvar}: {i+1}/{n_params} {'#'*40}")
        ders_dict[dvar] = cl_derivative_core.compute_cls_derivatives_wrt(dvar, **data)
        t2 = time.time()
        print(f"Elapsed time: {formatters.string_time_format(t2 - t1)}")


    tf = time.time()
    print("")
    print(f"Cl derivatives total elapsed time: {formatters.string_time_format(tf - ti)}")

    with open(cl_ders_file, mode="wb") as f:
        pickle.dump(ders_dict, f)
else:
    with open(cl_ders_file, mode="rb") as f:
        ders_dict = pickle.load(f)
        

### Plot of derivatives

In [ ]:
dcl_phph = ders_dict['w0'].dcl_dict['PhotometricGalaxy_PhotometricGalaxy']

for i in range(10):
    plt.plot(dcl_phph.l_bin_centers, dcl_phph.dc_lij[:, i, i], label=i+1)

plt.legend(bbox_to_anchor=[1.0, 0.4], title="bin index")
plt.title(r"Derivative of $C^{\rm phph}_{ii}(\ell)$ w.r.t. to $w_0$")
plt.xscale('log')
plt.xlabel(r"$\ell$", fontsize=22)
plt.ylabel(r"$\frac{\partial C^{\rm phph}_{ii}(\ell)}{\partial w_0}$", rotation=0, fontsize=22, labelpad=50)

## Compute and save Delta Cls

Compute the $\Delta C(\ell)$'s. These are simply defined as

\begin{equation}
\Delta C^{AB}_{ij}(\ell) = \frac{1}{\sqrt{f_{\rm sky}\Delta\ell}} \left[ C^{AB}_{ij}(\ell) + N^{AB}_{ij}(\ell) \right]
\end{equation}

where $N^{AB}_{ij}(\ell)$ is the Poisson shot noise, and $f_{\rm sky}$ is the sky fraction covered by Euclid. The sky fraction is computed assuming an observed sky area of $15000 \, \mathrm{deg}^2$, and therefore since the full sky is approximately $41252.961 \, \mathrm{deg}^2$ we have

\begin{equation}
f_{\rm sky} \simeq 0.36361
\end{equation}

The covariance matrix for the $C(\ell)$'s is calculated as

\begin{equation}
\mathrm{Cov}[ C^{AB}_{ij}(\ell), C^{CD}_{km}(\ell')] = \frac{\delta_{\ell\ell'}^{\rm K}}{2\ell + 1} 
\left[
\Delta C^{AC}_{ij}(\ell)\Delta C^{BD}_{ij}(\ell') + \Delta C^{AD}_{ij}(\ell) \Delta C^{BC}_{ij}(\ell')
\right]
\end{equation}

where $\delta_{\ell\ell'}^{\rm K}$ is the Kronecker delta.

The following cell computes and saves on disk the $\Delta C^{AB}(\ell)$'s for each combination of the probe indices $A$ and $B$.

In [ ]:
if not ws.delta_cls_file.is_file():
    delta_cls = cl_core.compute_delta_cls(ws)
    delta_cls.saveToHDF5(ws.delta_cls_file)
else:
    delta_cls = cl_core.DeltaClCollection.fromHDF5(ws.delta_cls_file)

## Fisher

### Theory
Here we compute the Fisher matrices. The formula for the computation of the Fisher matrix is

\begin{equation}
F_{\alpha \beta} = 
\sum_{\ell = \ell_{\rm min}}^{\ell_{\rm max}}
\frac{\partial \mathbf{C}(\ell)^{T}}{\partial \theta_{\alpha}} \mathrm{Cov}[\mathbf{C}(\ell), \mathbf{C}(\ell)]^{-1} \frac{\partial \mathbf{C}(\ell)}{\partial \theta_{\beta}}
\end{equation}

where $\mathbf{C}(\ell)$ is the datavector containing the angular power spectra involved in the computation of the Fisher matrix. Since the $C(\ell)$'s are matrices, it is necessary to turn them into vectors using $\mathrm{vecp}$ or $\mathrm{vec}$ depending on whether they are symmetric (auto-correlations) or not. 
As an example, if one wants to compute the Fisher matrix $[\mathrm{WL}+\mathrm{GC_{ph}}+\mathrm{XC}(\mathrm{WL},\mathrm{GC_{ph}})]$ the data-vector will be the following:

\begin{equation}
\mathbf{C}(\ell) = \left\{\mathbf{C}^{\rm wlwl}(\ell), \mathbf{C}^{\rm phph}(\ell), \mathbf{C}^{\rm wlph}(\ell)\right\}
\end{equation}

where $\rm wl \equiv WL$ and $\rm ph \equiv GC_{ph}$. The corresponding covariance matrix is a block matrix with the following layout:

\begin{equation}
\mathrm{Cov}[\mathbf{C}(\ell), \mathbf{C}(\ell)]=
\left(\begin{array}{lll}
\mathrm{Cov}\left[\mathbf{C}^{\mathrm{wlwl}}(\ell), \mathbf{C}^{\mathrm{wlwl}}(\ell)\right] & 
\mathrm{Cov}\left[\mathbf{C}^{\mathrm{wlwl}}(\ell), \mathbf{C}^{\mathrm{phph}}(\ell)\right] & 
\mathrm{Cov}\left[\mathbf{C}^{\mathrm{wlwl}}(\ell), \mathbf{C}^{\mathrm{wlph}}(\ell)\right] \\
\mathrm{Cov}\left[\mathbf{C}^{\mathrm{phph}}(\ell), \mathbf{C}^{\mathrm{wlwl}}(\ell)\right] & 
\mathrm{Cov}\left[\mathbf{C}^{\mathrm{phph}}(\ell), \mathbf{C}^{\mathrm{phph}}(\ell)\right] & 
\mathrm{Cov}\left[\mathbf{C}^{\mathrm{phph}}(\ell), \mathbf{C}^{\mathrm{wlph}}(\ell)\right] \\
\mathrm{Cov}\left[\mathbf{C}^{\mathrm{wlph}}(\ell), \mathbf{C}^{\mathrm{wlwl}}(\ell)\right] & 
\mathrm{Cov}\left[\mathbf{C}^{\mathrm{wlph}}(\ell), \mathbf{C}^{\mathrm{phph}}(\ell)\right] & 
\mathrm{Cov}\left[\mathbf{C}^{\mathrm{wlph}}(\ell), \mathbf{C}^{\mathrm{wlph}}(\ell)\right]
\end{array}\right)
\end{equation}

### Implementation

The needed input data for the computation are:

* the physical parameters involved in the forecast;
* the $\Delta C^{AB}(\ell)$'s;
* the derivatives of the $C^{AB}(\ell)$ for the free parameters of the forecast.

Also the **datavectors** for the Fisher computation must be specified. With datavector here we mean the information about which $C(\ell)$ are to be inserted in the particular Fisher matrix computation, with the definition given above. The representation of the datavector is given by strings separated by underscores. 

Here as an example we calculate the following Fisher matrices:

* $[\mathrm{WL}]$: `wlwl`;
* $[\mathrm{GC_{ph}}]$: `phph`;
* $[\mathrm{WL}+\mathrm{GC_{ph}}]$: `wlwl_phph`;
* $[\mathrm{WL}+\mathrm{GC_{ph}}+\mathrm{XC}(\mathrm{WL},\mathrm{GC_{ph}})]$: `wlwl_phph_wlph`.

The square brackets serves to identify the datavector extent. Hence with $[\mathrm{WL}+\mathrm{GC_{ph}}]$ we mean to combine the two probes *keeping the cross-covariance into account*.

Let's start the computation. First of all we compute the auto-correlation fishers

In [ ]:
ws.base_fishers_dir.mkdir(exist_ok=True)

fisher_input_data = {
    "outdir": ws.base_fishers_dir,
    "ws": ws,
    "phys_pars": phys_pars,
    "delta_cls": delta_cls,
    "dcoll_dvar_dict": ders_dict
}

auto_fishers = fisher_core.compute_and_save_fishers(["phph", "spsp", "wlwl"], ws.base_fishers_dir, ws, phys_pars, delta_cls, ders_dict)

Now compute some combinations

In [ ]:
brief_str_datavectors = [
    "wlwl_phph", "wlwl_phph_wlph", 
    "phph_spsp", "phph_spsp_phsp",
    "wlwl_spsp", "wlwl_spsp_wlsp",
    "wlwl_phph_wlph_spsp",
    "wlwl_phph_wlph_spsp_wlsp",
    "wlwl_phph_wlph_spsp_phsp",
    "wlwl_phph_wlph_spsp_wlsp_phsp",
]

fishers = fisher_core.compute_and_save_fishers(brief_str_datavectors, ws.base_fishers_dir, ws, phys_pars, delta_cls, ders_dict)

Load IST:F Pk fisher matrix

In [ ]:
from seyfert.fisher.fisher_matrix import FisherMatrix

f_gcsp_pk = FisherMatrix.from_ISTfile(filesystem_utils.get_ist_gcsp_pk_fisher_file("optimistic"))

## Some contour plots

Plot Fisher contours

In [ ]:
from seyfert.fisher.fisher_analysis import FisherAnalysis
from seyfert.fisher.fisher_plot import FisherPlotter

full_analysis = FisherAnalysis.fromFishersDir(ws.base_fishers_dir, params=phys_pars)

full_analysis.fisher_matrices.update({
    "[GCsp(Pk)]": f_gcsp_pk
})

cmap = matplotlib.colors.ListedColormap(["dodgerblue", "forestgreen", "gold"])
pars_to_plot=["Omm", "h", "w0", "wa"]

### WLxGCph

In [ ]:
wlph_analysis = full_analysis.getSubsetAnalysis(["[WL]", "[WL+GCph]", "[WL+GCph+XC(WL,GCph)]"])
wlph_analysis.evaluateMarginalizedErrors()
wlph_analysis.name = "WLxGCph"

plotter = FisherPlotter(pars_to_plot=pars_to_plot, fisher_analysis=wlph_analysis)
plotter.setPlotConfig(config_file="input/config/results_config.json")
plotter.setParametersPlotRanges()
plotter.config.cmap = cmap

fig, axs = plotter.makeTriangularPlot()

### WLxGCsp

In [ ]:
phsp_analysis = full_analysis.getSubsetAnalysis(["[WL]", "[WL+GCsp]", "[WL+GCsp+XC(WL,GCsp)]"])
phsp_analysis.evaluateMarginalizedErrors()
phsp_analysis.name = "WLxGCsp"

plotter = FisherPlotter(pars_to_plot=pars_to_plot, fisher_analysis=phsp_analysis)
plotter.setPlotConfig(config_file="input/config/results_config.json")
plotter.setParametersPlotRanges()
plotter.config.cmap = cmap

fig, axs = plotter.makeTriangularPlot()

### 6x2pt

In [ ]:
sixtwopt_analysis = full_analysis.getSubsetAnalysis([
    '[WL+GCph+XC(WL,GCph)]',
    '[WL+GCph+XC(WL,GCph)+GCsp]',
    '[WL+GCph+XC(WL,GCph)+GCsp+XC(WL,GCsp)]',
    '[WL+GCph+XC(WL,GCph)+GCsp+XC(GCph,GCsp)]',
    '[WL+GCph+XC(WL,GCph)+GCsp+XC(WL,GCsp)+XC(GCph,GCsp)]'
])

sixtwopt_analysis.evaluateMarginalizedErrors()
sixtwopt_analysis.name = "6x2pt"

plotter = FisherPlotter(pars_to_plot=pars_to_plot, fisher_analysis=sixtwopt_analysis)
plotter.setPlotConfig(config_file="input/config/results_config.json")
plotter.setParametersPlotRanges()

fig, axs = plotter.makeTriangularPlot()

In [ ]:
t_end = time.time()

print(f"Total notebook run time: {formatters.string_time_format(t_end - t_begin)}")